In [34]:


from typing import Annotated
from pydantic import BaseModel, EmailStr, Field, PositiveInt, StringConstraints, ValidationError


class Employee(BaseModel):
    age: PositiveInt
    email: EmailStr | None = None
    name: Annotated[str, StringConstraints(min_length=3, max_length=50, pattern="^P.*$")]
    salary: Annotated[float, Field(gt=100)]

peter = Employee(name="Peter", age=35, salary=1000.0)


In [29]:
try:
    john = Employee(name="John", age=35, salary=50.0)
except ValidationError as e:
    print(e)

2 validation errors for PersonWithValidation
name
  String should match pattern '^P.*$' [type=string_pattern_mismatch, input_value='John', input_type=str]
    For further information visit https://errors.pydantic.dev/2.6/v/string_pattern_mismatch
salary
  Input should be greater than 100 [type=greater_than, input_value=50.0, input_type=float]
    For further information visit https://errors.pydantic.dev/2.6/v/greater_than


In [35]:
from pydantic import field_validator, model_validator


class UserModel(BaseModel):
    name: str
    surname: str

    @field_validator('name')
    @classmethod
    def name_must_start_with_capital_letter(cls, v: str) -> str:
        if not v[0].isupper():
            raise ValueError('name must start with a capital letter')
        return v

    @model_validator(mode='after') # type: ignore
    def name_and_surname_must_be_different(self) -> 'UserModel':
        if self.name == self.surname:
            raise ValueError('name and surname must be different')
        return self

user_model_correct = UserModel(name="Peter", surname="Smith")
print(user_model_correct)


name='Peter' surname='Smith'


In [32]:
try:
    user_model_incorrect = UserModel(name="Peter", surname="Peter")
except ValidationError as e:
    print(e)

1 validation error for UserModel
  Value error, name and surname must be different [type=value_error, input_value={'name': 'Peter', 'surname': 'Peter'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.6/v/value_error
